In [1]:
import json
import os
import time
import pandas as pd

from amazon_scraper import AmazonScraper, ScrapingConfig

chunk_size = 10


# Function to split DataFrame into chunks
def split_dataframe(df: pd.DataFrame, chunk_size: int) -> list:
    return [df.iloc[i : i + chunk_size] for i in range(0, len(df), chunk_size)]


series = pd.read_pickle("./data/pfw/03_sample_asins.pkl")

if not isinstance(series, pd.Series):
    raise Exception("df is not a pd.Series")

df = pd.DataFrame(
    columns=["asin", "review_complete"],
    data={"asin": series, "review_complete": [None] * len(series)},
)

df = df.sort_index()

# Split the DataFrame into chunks
chunks = split_dataframe(df, chunk_size)

config = ScrapingConfig(
    max_pages=10,
    max_workers=10,
    # max_concurrent_requests=3,
    request_timeout=30,
    retry_attempts=3,
)

start_time = time.time()

asins = chunks[0]["asin"]

async with AmazonScraper(config) as scraper:
    results = await scraper.scrape_asins(asins)
    for product in results:
        # Ensure the directory exists
        output_dir = "./data/pfw/results/"
        os.makedirs(output_dir, exist_ok=True)

        # Write product data to a JSON file
        file_path = os.path.join(output_dir, f"{product['asin']}.json")
        with open(file_path, "w") as json_file:
            json.dump(product, json_file, indent=4)

        print(f"File successfully created: {file_path}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"time elapsed {elapsed_time}")

Found 1068 unique reviews for ASIN B0043T7FXE
Error fetching https://www.amazon.com/product-reviews/B07B7K7N3P?sortBy=recent&pageNumber=9&filterByStar=positive&formatType=all_formats&mediaType=media_reviews_only: 
Error fetching https://www.amazon.com/product-reviews/B07B7K7N3P?sortBy=recent&pageNumber=10&filterByStar=positive&formatType=all_formats&mediaType=media_reviews_only: 
Error fetching https://www.amazon.com/product-reviews/B07B7K7N3P?sortBy=recent&pageNumber=1&filterByStar=positive&formatType=all_formats&mediaType=all_contents: 
Error fetching https://www.amazon.com/product-reviews/B07B7K7N3P?sortBy=recent&pageNumber=2&filterByStar=positive&formatType=all_formats&mediaType=all_contents: 
Error fetching https://www.amazon.com/product-reviews/B07B7K7N3P?sortBy=recent&pageNumber=3&filterByStar=positive&formatType=all_formats&mediaType=all_contents: 
Error fetching https://www.amazon.com/product-reviews/B07B7K7N3P?sortBy=recent&pageNumber=3&filterByStar=one_star&formatType=curren